# Welcome future python subsurface professionals! (yes, you!)


< Operational note: This works best in Firefox or Chrome, do NOT use edge or internet expolorer, netscape naviator, AOL, etc. >

# Welcome to Colab!

Google Colab is a online, web hosted way to work through Jupyter Notebooks. They have a pro version that adds larger GPU's and more RAM for $10 USD a month. For today, the free version will work great. It can be accessed on any computer with an internet connection and a modern browser. 

#### Mounting Colab to Google Drive

The hardest part of Colab is getting the google drive mounted and finding your file locations. Google does this to keep your data 'safe' but does have some drawbacks. Because it is an ipython notebook, you can use the tricks of cd, ls, pwd, and others to navigate to your folder of interest.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

"ls" is the list command. 

In [0]:
ls

My folder name is t20-intro, you might need to change it to match your folder name.

In [0]:
cd gdrive/My Drive/denver-data/

pwd is the command to display the path to your location

In [0]:
pwd

In [0]:
ls

 Unix cheet sheet is [here](https://files.fosswire.com/2007/08/fwunixref.pdf). I am not going to go into detail but it's good to know a few key ones!

# Python!

Finally! Let's get into why y'all are here. Python has some basic math functions built in:



In [0]:
2 + 2

In [0]:
3 * 11.1

In [0]:
print('hello!')

In [0]:
a = 5
b = 2.1

a * b 

In [0]:
# This is a code cell, but a # makes it a comment
2 * 2 + 3

python indexes 0,1,2,3 not 1,2,3,4 (like matlab)

In [0]:
c = 'rad'
c[0]

In [0]:
c[1]

There are whole classes devoted to basic python syntax, loops, functions, etc. This class is to just get you up and running with some key basics that can help with basic projects. Join us on the slack for some pointers!

## Importing and Using Packages

Packages are similar to Matlab toolboxes. You import them in a line of code in your notebook. 

In [0]:
import matplotlib.pyplot as plt

Ok, so we are telling the computer everytime we want to use the pyplot function from matplotlib, and we are going to call it plt as it's way shorter. Programmers are lazy. 

In [0]:
import pandas as pd
import numpy as np

Importing a few others that I use in (almost) every project. We can import them normally becuase Google pre-loaded them. Google does not pre-load everything that you need, sometimes you might need to install it.  ~*warning* ~ not every package can easily be imported in Colab. I have not figured out what makes some go and some not. But stats and plotting is usually pretty safe. Niche, one off, old, academic code, less so. 

In [0]:
!pip install lasio # the ! before the pip is important

Just installed the lasio package using pip. Lasio is a .las file imput and output reader. It's a great way to read in a .las file (well logs, not LiDAR) to use in python. After installing it, you still need to import it. The github for lasio is [here](https://github.com/kinverarity1/lasio)

In [0]:
import lasio

There are hundreds of packages, not all of them work with Colab. We will install and import packages throughout the notebook.

# Well log input and output using LasIO

LasIO (.las input and output) is one of my all time favorite packages. It's updated often, and it works 99.5% of the time with .las files (well logs, not LiDAR data). The github is [here](https://github.com/kinverarity1/lasio). A quick note is all of this is just examples, no promises of petrophysical sense!

In [0]:
las = lasio.read('4900722147_722147B.las')

We just read in a las file! Let's do a quick quaility control, and make some plots.

In [0]:
las.curves

Wow, this well has a lot of curves! OK, we are going to make a data frame for the data in the .las file. This will behave really similar to the previous data frame

In [0]:
data = las.df() # This time we named the dataframe data and not df
data.head(3)

Scroll around! If you want to get a handle on stats of your well curves, using the describe function on a dataframe is awesome. 

In [0]:
data.to_csv('well_data.csv') # just saved it out!

Once this is in a data frame, you can export the curve data as a csv! This can be used in excel, spotfire, matlab or any other program you are more familiar with. This alone saves tons of time! While I prefer python, I know it's not for everyone.

In [0]:
data.describe()

If you want to save out this table as a csv, you can do that in one line:

In [0]:
data.describe().to_csv("petro-stats.csv")

In [0]:
data.AHO10.min() # If you just want one of the stats

Let's use the quantile function to make some quick cutoffs (will be used in the future).

In [0]:
lowGR = data.GR.quantile(.20)
lowGR

In [0]:
highRES = data.AHO90.quantile(.95)
highRES

Feel free to make your own cutoffs here!

### Plotting up well log data

In [0]:
# Cross plot comparing GR to Deep Resistivity
plt.figure(figsize=(5,4), dpi=100) # figure size and dpi you can set here
plt.scatter(data.GR.values, data.AHO90.values, color='blue', marker='.', alpha=0.3)
plt.yscale('log') #log scale for Y axis

plt.grid(True)
plt.xlabel('GR', size=16)
plt.ylabel('Deep Resistivity', size=16)

In [0]:
# lets make a histogram of the GR Curve, with 20 bins, in green
plt.hist(data.GR.values, bins = 20, color='g', alpha = 0.5)
plt.xlabel('GR', size=16)
plt.ylabel('Count', size=16)

In [0]:
# line plot of the Gamma curve
plt.figure(figsize=(4,8), dpi=100)
plt.plot(data.GR.values, data.index, color='g')
plt.ylabel('Depth in Feet', size=16)
plt.ylim(2000,0)
plt.grid(True)


Let's add a depth track for deep resistivity, a title and a few other additional widigits. Also we will use the well name for the title:

In [0]:
las.header['Well'].WELL.value # we can pull the well name directly from the header

In [0]:
# line plot of the Gamma curve
plt.figure(figsize=(6,7), dpi=100)

plt.suptitle(las.header['Well'].WELL.value, size =16) # overall title

plt.subplot(121) # if we are going to make two plots, matplotlib calls it a subplot
plt.plot(data.GR.values, data.index, color='g') # the actual plot!
plt.ylabel('Depth in Feet', size=16)
plt.xlabel('API', size=16)
plt.ylim(2000,0) # the limit is reversed to go deep down
plt.grid(True) # Turning the grid on

plt.subplot(122)
plt.plot(data.AHO10.values, data.index, color='grey')
plt.plot(data.AHO90.values, data.index, color='black')
plt.xscale('log')
plt.grid(True)
plt.xlim(0.4,4000)
plt.ylim(2000,0)
plt.xlabel('OhmM', size=16)
plt.tick_params(labelleft=False)  

plt.savefig('awesome-plot.pdf', dpi=200)

Let's add a pay flag, remember the cutoffs we calcualted above? Let's use those. Using pythonic nomenclature, we will set a true flag, only when both conditions are met.

In [0]:
payMapper = (data.GR.values <= lowGR) & (data.AHO90.values >= highRES)
np.unique(payMapper)

If we want to see how many true and false statements there are:

In [0]:
np.bincount(payMapper)

So the GR values have to be below the cutoff, and resistivity have to be above the cutoff.

In [0]:
# line plot of the Gamma curve
plt.figure(figsize=(5,8), dpi=100)

plt.subplot(131) # the subplot changed to allow for 3 columns
plt.plot(data.GR.values, data.index, color='g')
plt.ylabel('Depth in Feet', size=16)
plt.xlabel('API', size=16)
plt.ylim(2000,0)
plt.grid(True)

plt.subplot(132) # this is the new pay flag plot
plt.fill_between(payMapper, data.index, color='red') # used fill between, not plot
plt.ylim(2000,0)
plt.tick_params(labelleft=False)  
plt.xlabel('Pay Flag', size=16)
plt.grid(True)

plt.subplot(133)
plt.plot(data.AHO90.values, data.index, color='black')
plt.xscale('log')
plt.grid(True)
plt.xlim(0.4,4000)
plt.ylim(2000,0)
plt.xlabel('OhmM', size=16)
plt.tick_params(labelleft=False)  



Little heavy on the pay flag! But a good template for future work. Can be used for any sort of cutoff, or statistical analysis. Mess around with a plot below!

# Closing thoughts / hot takes

* It has been and always will be a data problem
* For subsurface workflows, Python >>> *, so many packages written for subsurface geodata. Segyio, Verde, Welly, Striplog, all come to mind. 
* Not every problem requires high dollar machine learning, but usually every problem requires good data
* living with uncertainty is the role of subsurface G&G work
* Most (smaller) companies would be better served by hiring a part time developer/consultant to make great tools using open source software compared to paying for licenses